In [2]:
from pyspark.sql import SparkSession
import glob
import numpy as np
import pandas as pd
import os
import concurrent.futures
from tqdm import tqdm

In [3]:
#Set Environment accordingly
os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home/"
os.environ['SPARK_HOME'] = "/Users/simran/Downloads/spark-3.5.5-bin-hadoop3/"

In [4]:
directory_path = "./full_history"
file_pattern = "*.csv"

In [5]:

# Initialize Spark session
spark = SparkSession.builder \
    .appName("BigDataProject1") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.memory", "12g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "20g") \
    .config("spark.default.parallelism", "100") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC") \
    .config("spark.serializer", "org.apache.spark.serializer.JavaSerializer") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/25 14:51:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType

schema = StructType([
    StructField("date", DateType(), True),
    StructField("volume", DoubleType(), True),  
    StructField("open", DoubleType(), True),
    StructField("high", DoubleType(), True),
    StructField("low", DoubleType(), True),
    StructField("close", DoubleType(), True),
    StructField("adj close", DoubleType(), True),
    StructField("StockName", StringType(), True)
])

In [7]:
# Load datasets as DataFrames
history_df = spark.read.format("csv") \
     .option("header", "true") \
     .option("treatEmptyValuesAsNulls", "true") \
     .option("schema",schema) \
     .load(f"{directory_path}/{file_pattern}")


history_df.show(5)

+----------+-------+-----------------+------------------+------------------+------------------+------------------+---------+
|      date| volume|             open|              high|               low|             close|         adj close|StockName|
+----------+-------+-----------------+------------------+------------------+------------------+------------------+---------+
|2023-12-28|5703600|30.14999961853028|30.239999771118164|29.950000762939453| 30.18000030517578| 30.18000030517578|      HPQ|
|2023-12-27|5127900|30.39999961853028|30.520000457763672|30.209999084472656|30.239999771118164|30.239999771118164|      HPQ|
|2023-12-26|4853600|30.13999938964844|30.549999237060547|30.059999465942383| 30.40999984741211| 30.40999984741211|      HPQ|
|2023-12-22|4476400|30.21999931335449|30.479999542236328| 30.06999969482422| 30.18000030517578| 30.18000030517578|      HPQ|
|2023-12-21|6529200|30.07999992370605| 30.21999931335449| 29.90999984741211| 30.13999938964844| 30.13999938964844|      HPQ|


In [8]:
# Remove duplicates
history_df = history_df.dropDuplicates()

# Handle missing values for both open and close simultaneously
history_df = history_df.na.fill({
    "open": np.nan,
    "close": np.nan
})
history_df = history_df.replace(float('nan'), np.nan)

history_df.cache()

DataFrame[date: string, volume: string, open: string, high: string, low: string, close: string, adj close: string, StockName: string]

In [ ]:
history_df.describe().show

In [ ]:
history_df.count()

In [9]:
#Convert Data to Pandas DataFrame to plot visualisations

#all_files = glob.glob("./full_history/*.csv")

#history_df_pd = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

history_df_pd = history_df.toPandas()

25/04/25 15:05:09 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks 
java.io.IOException: Connecting to mac/192.168.1.222:61371 failed in the last 4750 ms, fail this connection directly
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:210)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.start(RetryingBlockTransferor.java:152)
	at org.apache.spark.network.netty.NettyBlockTransferService.fetchBlocks(NettyBlockTransferService.scala:151)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:102)
	at org.apache.spark.storage.BlockManager.fetchRemoteManagedBuffer(BlockManager.scala:1172)
	at org.apache.spark.storage.Bloc

Py4JJavaError: An error occurred while calling o57.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 8.0 failed 1 times, most recent failure: Lost task 4.0 in stage 8.0 (TID 15852) (mac executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGSchedu

25/04/25 15:06:23 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks (after 1 retries)
java.io.IOException: Connecting to mac/192.168.1.222:61371 failed in the last 4750 ms, fail this connection directly
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:210)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.lambda$initiateRetry$0(RetryingBlockTransferor.java:206)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.jav

In [17]:
#Data Visualisations

#Time series visualisation given stock name

import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.dates as mdate
from IPython.display import display


# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=list(history_df_pd['StockName'].unique()),
    description='StockNames:'
)

# Function to update the plot based on dropdown selection
def update_plot(category):
    plt.clf() # Clear previous plot
    filtered_df = history_df_pd[history_df_pd['StockName'] == category]
    filtered_df.loc[:,'date'] = pd.to_datetime(filtered_df['date'])
    # Plotting closing prices for a single stock
    sorted_filtered_df = filtered_df.sort_values(by='date')
    plt.plot(sorted_filtered_df['date'], sorted_filtered_df['close'])
    plt.title(f'Plot for {category}')
    plt.xlabel('Time')
    plt.ylabel('Price')
    locator = mdate.YearLocator()
    plt.gca().xaxis.set_major_locator(locator)

    plt.gcf().autofmt_xdate()


# Observe the dropdown value and update the plot
out = widgets.interactive_output(update_plot, {'category': dropdown})

# Display the dropdown and the initial plot

display(dropdown, out)

Dropdown(description='StockNames:', options=('RIV', 'ANTE', 'CSCO', 'PRI', 'NZF', 'HLNE', 'HUBS', 'GPL', 'PSM'…

Output()

In [18]:
#Data Visualisations

#Moving Averages visualisation given stock name


# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=list(history_df_pd['StockName'].unique()),
    description='StockNames:'
)

# Function to update the plot based on dropdown selection
def update_plot(category):
    plt.clf() # Clear previous plot
    filtered_df = history_df_pd[history_df_pd['StockName'] == category]
    filtered_df.loc[:,'date'] = pd.to_datetime(filtered_df['date'])
    # Plotting closing prices for a single stock
    sorted_filtered_df = filtered_df.sort_values(by='date')
    ma100 = sorted_filtered_df.close.rolling(100).mean()
    plt.plot(sorted_filtered_df['date'], sorted_filtered_df['close'])
    plt.plot(sorted_filtered_df['date'],ma100, 'r')
    plt.title(f'Plot for {category}')
    plt.xlabel('Time')
    plt.ylabel('Price')
    locator = mdate.YearLocator()
    plt.gca().xaxis.set_major_locator(locator)

    plt.gcf().autofmt_xdate()


# Observe the dropdown value and update the plot
out = widgets.interactive_output(update_plot, {'category': dropdown})

# Display the dropdown and the initial plot

display(dropdown, out)

Dropdown(description='StockNames:', options=('RIV', 'ANTE', 'CSCO', 'PRI', 'NZF', 'HLNE', 'HUBS', 'GPL', 'PSM'…

Output()